# PCA Algorithm to distinguish different hand signs extended with K-Means
By Mailin Brandt and Finian landes

In [66]:
### Imports
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [48]:
#Loads Images from dir, size has to be provided (image in format size x size)
#returns images as vectors and image names as list
def load_images(dir: str, size: int = 256, grayscale: bool = True) -> np.ndarray:
    image_names: list = sorted(os.listdir(dir))
    n: int = len(image_names)
    images: np.ndarray = np.zeros([n, size ** 2])
    for i, name in enumerate(image_names):
        img: Image = Image.open(dir + "//"+ name)
        images[i] = img.getdata(0)
    return images, np.array(image_names)

#Generates a random train and test set with n_test test items
#returns train and test images and train and test names
def get_train_test_dataset(images: np.ndarray, image_names: list[str], n_test: int) -> tuple[np.ndarray, np.ndarray, list, list]:
    indices: list = np.random.choice(len(images), n_test, replace = False)
    test: np.ndarray = images[indices]
    names_test: list[str] = image_names[indices]
    train: np.ndarray = np.delete(images, indices, axis = 0)
    names_train: list[str] = np.delete(image_names, indices)
    return train, test, names_train, names_test

#Using the coeff_matrix, the eigenfaces and the meanface returns the index of the image which lies closest to point
#Returns index of closest image
def closest_neighbour(point: np.ndarray, meanface: np.ndarray , eigenfaces: np.ndarray, coeff_mat: np.ndarray) -> int:
    c1: np.ndarray = np.matmul(eigenfaces, point - meanface)
    distances: np.ndarray = np.sum((c1 - coeff_mat.T)**2, axis=1)
    return np.argmin(distances)

#Creates a matrix from the meanface, the eigenfaces and the passed points, which is used to increase efficiency of the PCA-Algorithm
#Returns a matrix in form n x n where n is the number of images
def coeff_matrix(meanface: np.ndarray, eigenfaces: np.ndarray, points: np.ndarray) -> np.ndarray:
    return np.matmul(eigenfaces, (points - meanface).T)

#Saves an image with a given filename stable for image in any format 
#Returns None
def save_image(picture: np.ndarray, filename: str, location: str = "c:/Users/finia/OneDrive - SBL/PrA/PCA of hand signs/Processed Images/Eigenfaces", size: int = 256) -> None:
    min_val: float = np.min(picture)
    picture = picture - min_val
    picture = picture / np.max(picture)
    picture = (255 * picture).astype(np.uint8)
    n_picture = np.reshape(picture, (size, size)).astype(dtype = np.uint8)
    plt.imshow(n_picture, cmap = "gray")
    plt.show()
    im = Image.fromarray(n_picture, mode="L")
    im.save(location + "/" + filename + ".jpg")

#Initializes centroids for K-Means by choosing k random points
#Returns k centroids
def init_centroids(k: int, points: np.ndarray) -> np.ndarray:
    centroids: np.ndarray = points[np.random.choice(len(points), k, replace = False)]
    return centroids

#Run K-means
#Returns new centroids and the classification of the points
def run_iteration(points: np.ndarray, centroids: np.ndarray, k: int) -> tuple[np.ndarray, list]:
    classification: list = []
    clusters: list[np.ndarray] = [[] for _ in range(k)]
    new_centroids: np.ndarray = np.zeros((k, points.shape[1]))

    for point in points:
        distances: list[np.ndarray] = [np.linalg.norm(point - centroid) for centroid in centroids]
        classification.append(np.argmin(distances))
        clusters[classification[-1]].append(point)
    for i in range(len(centroids)):
        if len(clusters[i]) == 0:
            new_centroids[i] = points[np.random.randint(0, points.shape[0])]
        else:
            new_centroids[i] = np.mean(clusters[i], axis = 0)
    return classification, new_centroids

#Checks whether k-means is converged by comparing the distances between the old and the new centroids
#Returns True when K-means is NOT converged
def is_not_converged(last_centroids: np.ndarray, current_centroids: np.ndarray, e: float = 1e-20) -> bool:
    dist: float = 0
    for  i in range(len(last_centroids)):
        dist += np.sqrt(np.sum((last_centroids[i] - current_centroids[i]) ** 2))
    return dist > e

#Clusters the names of the images using the classification of the points
#Returns list with the names of images in each cluster, where each cluster is depicted by a sublist
def cluster_names(classification: list, names: list, k: int) -> list:
    classified_names: list = [[] for _ in range(k)]
    for i, name in enumerate(names):
        classified_names[classification[i]].append(name)
    return classified_names

#Computes coefficients from a point a mean and a point array
#Returns those coefficients as np.array
def compute_coefficients(point: np.ndarray, points: np.ndarray, mean: np.ndarray) -> np.ndarray:
    return np.matmul(points, point - mean)

#Calculates the distance between a point and a mean by additionally using a points array to make a transformation in space to make the eclidean distance more meaningful
#Returns the distance as float
def distance(point: np.ndarray, q: np.ndarray, mean: np.ndarray, points: np.ndarray) -> float:
    c1: np.ndarray = compute_coefficients(point, points, mean)
    c2: np.ndarray = compute_coefficients(q, points, mean)
    return np.sqrt(np.sum((c1 - c2)**2))

#Calculates the distance between each point and a given center point 
#Returns the distances of each point to the center point
def outlying_distances(center: np.ndarray, points: np.ndarray) -> list:
    dist: list = [distance(point, np.zeros_like(point), center, points) for point in points]
    return dist

#Sorts points by taking their distance to a given center and then ordering them in ascending order 
#Returns the Distances and the sorted points aswell as the sorted names
def sort_points_names_by_dist(center: np.ndarray, points: np.ndarray, names: list) -> tuple[np.ndarray, np.ndarray, list]:
    distances: list = np.array(outlying_distances(center, points))
    new_names: np.ndarray = np.array(names)
    sorted_indicies: list = np.argsort(distances)
    distances: list = list(distances[sorted_indicies])
    new_names: list = list(new_names[sorted_indicies])
    points: np.ndarray = points[sorted_indicies]
    return distances, points, new_names

#Creats a count of how often each kind appears in each cluster
#Returns a list with dicts containig the count of items and also a score where 100% means that all clusters only contain one kind and 0% means that all kinds are distributed evenly
def k_means_score(cluster_names: list) -> tuple[list, float]:
    cluster_names: list = [[name[:-7] for name in sublist] for sublist in cluster_names]
    all_names: list = [item for sublist in cluster_names for item in sublist]
    without_duplicates: list = list(dict.fromkeys(all_names))
    result: list = []
    for sublist in cluster_names:
        count_dict: dict = {name: sublist.count(name) for name in without_duplicates}
        result.append(count_dict)
    total_items: int = len(all_names)
    max_counts: list = [max(cluster.values()) for cluster in result]
    actual_score: int = sum(max_counts)
    normalized_score: float = round(actual_score / total_items, 2)
    return result, normalized_score


In [69]:
image_dir_train: str = "c:/Users/finia/OneDrive - SBL/PrA/PCA of hand signs/Processed Images/Train/"
image_dir_test: str = "c:/Users/finia/OneDrive - SBL/PrA/PCA of hand signs/Processed Images/Test"
n_test_images: int = 50
train, names_train = load_images(image_dir_train)
test, names_test = load_images(image_dir_test)
#train, test, names_train, names_test = get_train_test_dataset(images, image_names, n_test_images)
k: int = 10
train = (train - np.mean(train)) / np.std(train)
dimensions: np.ndarray = train.shape[1]

In [64]:
meanface: np.ndarray = np.mean(train, axis=0)
A: np.ndarray = train - meanface
eigenfaces, s, VT = np.linalg.svd(A.transpose(), full_matrices=False)
eigenfaces: np.ndarray = eigenfaces.transpose()
coeff_mat: np.ndarray = coeff_matrix(meanface, eigenfaces, train)

In [23]:
print(sort_points_names_by_dist(np.mean(train),train,names_train))
"""
avg = sum(distances) / len(distances)
print(round(statistics.pstdev(distances) / statistics.mean(distances) * 100, 5))
print(list(zip(distances, new_names)))
"""
#Lower 1000
#Upper 5000

([19602337232.73344, 19967675820.473194, 20015205890.832, 20049949967.086365, 20089328123.978493, 20413941931.698296, 20417921076.710064, 20443741178.150227, 20470932782.513466, 20490895764.11545, 20499052184.415405, 20535007156.28178, 20549845550.48775, 20603998144.69826, 20646856044.683372, 20680857772.433777, 20684516876.836975, 20705775729.630905, 20713050902.87245, 20726188079.8666, 20754859820.808094, 20760526997.143196, 20781875312.310204, 20793546532.973766, 20794401984.986923, 20839231718.355442, 20849455311.127705, 20880038779.223328, 20885943980.233097, 20891531207.648422, 20902948679.056435, 20912186245.309055, 20946664763.642006, 20954886002.40061, 20954924799.7072, 20962508371.354263, 20970877187.11184, 20976982762.218357, 20984502699.90809, 21009977212.03263, 21027464092.22365, 21046833646.175247, 21054109251.51394, 21058008911.291985, 21063211448.41619, 21063753786.873314, 21074925125.694473, 21084950774.809174, 21115160680.939156, 21117632185.81294, 21142321109.15213, 

'\navg = sum(distances) / len(distances)\nprint(round(statistics.pstdev(distances) / statistics.mean(distances) * 100, 5))\nprint(list(zip(distances, new_names)))\n'

In [71]:
points: np.ndarray = np.matmul(coeff_mat, eigenfaces)
classification: list = []
centroids: np.ndarray = init_centroids(k, points)
init_cent: np.ndarray = centroids.copy()
last_centroids: np.ndarray = np.zeros_like(centroids)
i = 0

while is_not_converged(last_centroids, centroids):
    i += 1
    last_centroids = centroids.copy()
    classification, centroids = run_iteration(points, centroids, k)



kmeans: KMeans = KMeans(n_clusters=k, init=init_cent, random_state=42)
classification_sklearn: list = kmeans.fit_predict(points)
centroids_skearn: np.ndarray = kmeans.cluster_centers_


cluster_n: list = cluster_names(classification, names_train, k)
distribution, score = k_means_score(cluster_n)

cluster_n_sklearn: list = cluster_names(classification_sklearn, names_train, k)
distribution_sklearn, score_sklearn = k_means_score(cluster_n_sklearn)

print("K-Means score: " + str(score * 100) + "%")
print("K-Means score sklearn: " + str(score_sklearn * 100) + "%")



C:\Users\finia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1307: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)


K-Means score: 23.0%
K-Means score sklearn: 23.0%
